# Panel Regression - Firm Characteristics

A panel regression is a suitable regression method for analyzing our data, especially since you have quarterly data for multiple companies over a long period of time. A panel regression model accounts for both within-entity and between-entity variations, making it a useful tool to analyze data with a time series and cross-sectional dimension.  
With panel data, we can control for individual-level characteristics that may affect forecast accuracy by including fixed effects for each company or industry. We can also account for time-specific factors that may affect forecast accuracy by including time fixed effects or time-varying covariates.

### Random Effects Panel Regression

In [8]:
import pandas as pd
import numpy as np
import datetime as dt
import sklearn
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects


Data

In [9]:
df = pd.read_csv("Dataframes/characteristics_regression.csv")
df["Date"] = pd.to_datetime(df["Date"])
df

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,Earnings Per Share - Actual Surprise AbsVals,Revenue - Actual,Market Capitalization,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume,loss firm status
0,AVY.N,2013-01-01,Materials,11.178,11.178,1.532200e+09,3.426001e+09,0.799347,10.637479,207295.0,-1
1,AVY.N,2013-04-01,Materials,2.482,2.482,1.498900e+09,4.309428e+09,0.895787,24.148726,0.0,-1
2,AVY.N,2013-07-01,Materials,1.068,1.068,1.552300e+09,4.258229e+09,0.906078,-0.054218,0.0,-1
3,AVY.N,2013-10-01,Materials,8.095,8.095,1.504900e+09,4.273414e+09,0.903176,2.467620,236291.0,-1
4,AVY.N,2014-01-01,Materials,1.471,1.471,1.583900e+09,4.877168e+09,0.920659,16.003852,157761.0,-1
...,...,...,...,...,...,...,...,...,...,...,...
19196,POOL.OQ,2021-10-01,Retailing,17.194,17.194,1.411448e+09,1.742020e+10,3.684400,-5.130322,157740.0,-1
19197,POOL.OQ,2022-01-01,Retailing,40.267,40.267,1.035557e+09,2.268979e+10,4.504382,30.489705,82349.0,-1
19198,POOL.OQ,2022-04-01,Retailing,34.342,34.342,1.412650e+09,1.696712e+10,3.422880,-25.160664,290897.0,-1
19199,POOL.OQ,2022-07-01,Retailing,1.503,1.503,2.055818e+09,1.405795e+10,2.749481,-21.661570,185584.0,-1


Standardising and log transformation

In [10]:
rescale = df.copy()
rescale["Revenue - Actual"] = StandardScaler().fit_transform(np.array(rescale["Revenue - Actual"]).reshape(-1,1))
rescale["Market Capitalization"] = StandardScaler().fit_transform(np.array(rescale["Market Capitalization"]).reshape(-1,1))
rescale["Enterprise Value To Sales (Daily Time Series Ratio)"] = StandardScaler().fit_transform(np.array(rescale["Enterprise Value To Sales (Daily Time Series Ratio)"]).reshape(-1,1))
rescale["3 Month Total Return"] = StandardScaler().fit_transform(np.array(rescale["3 Month Total Return"]).reshape(-1,1))
rescale["Volume"] = StandardScaler().fit_transform(np.array(rescale["Volume"]).reshape(-1,1))

#log transform DV
rescale["Earnings Per Share - Actual Surprise AbsVals"] = np.log(rescale["Earnings Per Share - Actual Surprise AbsVals"] + 1)

df_clean = rescale
df_clean

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,Earnings Per Share - Actual Surprise AbsVals,Revenue - Actual,Market Capitalization,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume,loss firm status
0,AVY.N,2013-01-01,Materials,11.178,2.499631,-0.375130,-0.391005,-0.385356,0.433789,-0.290490,-1
1,AVY.N,2013-04-01,Materials,2.482,1.247607,-0.377964,-0.383696,-0.376219,1.321869,-0.356378,-1
2,AVY.N,2013-07-01,Materials,1.068,0.726582,-0.373419,-0.384120,-0.375244,-0.268964,-0.356378,-1
3,AVY.N,2013-10-01,Materials,8.095,2.207725,-0.377453,-0.383994,-0.375519,-0.103207,-0.281274,-1
4,AVY.N,2014-01-01,Materials,1.471,0.904623,-0.370730,-0.378999,-0.373863,0.786515,-0.306235,-1
...,...,...,...,...,...,...,...,...,...,...,...
19196,POOL.OQ,2021-10-01,Retailing,17.194,2.901092,-0.385406,-0.275230,-0.112024,-0.602611,-0.306241,-1
19197,POOL.OQ,2022-01-01,Retailing,40.267,3.720063,-0.417395,-0.231634,-0.034338,1.738654,-0.330204,-1
19198,POOL.OQ,2022-04-01,Retailing,34.342,3.565072,-0.385304,-0.278978,-0.136801,-1.919184,-0.263918,-1
19199,POOL.OQ,2022-07-01,Retailing,1.503,0.917490,-0.330569,-0.303046,-0.200599,-1.689192,-0.297391,-1


In [11]:
df_panel = df_clean.copy()

#### Fixed Effects Panel Regression

In a fixed effects panel regression, the individual-specific effects are modeled as fixed variables that do not vary across time. This means that the coefficients of the independent variables are estimated based on the within-entity variation in the data, which eliminates the effect of time-invariant unobserved heterogeneity.

Fixed effects models are useful when there are time-invariant unobserved variables that may affect the dependent variable, but are not included in the model. By modeling the individual-specific effects as fixed variables, fixed effects models can control for this unobserved heterogeneity and estimate the coefficients of the independent variables based on the within-entity variation, which provides more efficient estimates of the coefficients.

One limitation of fixed effects models is that they do not allow for testing the effect of time-invariant variables on the dependent variable. In addition, fixed effects models may suffer from the incidental parameter problem, which may lead to biased estimates of the coefficients of the independent variables in the presence of a large number of fixed effects.

In [12]:
#reformatting indices of dataframe for panel regression
df_clean['Instrument'] = df_clean['Instrument'].astype('category')
df_clean['Date'] = pd.to_datetime(df_clean['Date'])

# set the index to be the time variable and the cross-sectional variable
df_clean.set_index(['Instrument', 'Date'], inplace=True)
#df_clean

Regression with **Absolute** Surprise Values

In [13]:
y = df_clean.loc[:, "Earnings Per Share - Actual Surprise AbsVals"]
X = df_clean.loc[:, "Revenue - Actual":]

# perform the fixed effects panel regression
fixed_effects_model = PanelOLS(y, X, entity_effects=True, time_effects=True)

# fit the model and print the summary statistics
fixed_effects_results = fixed_effects_model.fit()
fixed_effects_results.summary


/Users/ginevrabramante/opt/anaconda3/envs/AnalystForecast/lib/python3.11/site-packages/linearmodels/panel/model.py:1183: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


Dep. Variable:,Earnings Per Share - Actual Surprise AbsVals,R-squared:,0.0081
Estimator:,PanelOLS,R-squared (Between):,-0.1076
No. Observations:,19081,R-squared (Within):,0.0098
Date:,"Tue, Mar 07 2023",R-squared (Overall):,-0.0925
Time:,20:19:03,Log-likelihood,-2.5e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,25.310
Entities:,502,P-value,0.0000
Avg Obs:,38.010,Distribution:,"F(6,18534)"
Min Obs:,2.0000,,
Max Obs:,40.000,F-statistic (robust):,25.310


#### By Industry: Example Tech Industry

With Absolute Surprise Values

In [14]:
group = df_panel[df_panel['GICS Industry Group Name'] == 'Technology Hardware & Equipment']

group['Instrument'] = group['Instrument'].astype('category')
group['Date'] = pd.to_datetime(group['Date'])
group.set_index(['Instrument', 'Date'], inplace=True)

y = group.loc[:, "Earnings Per Share - Actual Surprise AbsVals"]
X = group.loc[:, "Revenue - Actual":]

model = PanelOLS(y, X, entity_effects=True, time_effects=True).fit()
model

/var/folders/tj/07dch2wn75v90cdvrd8w0xm80000gn/T/ipykernel_69082/3204827800.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group['Instrument'] = group['Instrument'].astype('category')
/var/folders/tj/07dch2wn75v90cdvrd8w0xm80000gn/T/ipykernel_69082/3204827800.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group['Date'] = pd.to_datetime(group['Date'])
/Users/ginevrabramante/opt/anaconda3/envs/AnalystForecast/lib/python3.11/site-packages/linearmodels/panel/model.py:1183: MissingValueWarning: 
Inputs

Dep. Variable:,Earnings Per Share - Actual Surprise AbsVals,R-squared:,0.0281
Estimator:,PanelOLS,R-squared (Between):,-0.1188
No. Observations:,728,R-squared (Within):,0.0361
Date:,"Tue, Mar 07 2023",R-squared (Overall):,-0.0972
Time:,20:19:03,Log-likelihood,-842.14
Cov. Estimator:,Unadjusted,,
,,F-statistic:,3.1984
Entities:,19,P-value,0.0042
Avg Obs:,38.316,Distribution:,"F(6,664)"
Min Obs:,27.000,,
Max Obs:,40.000,F-statistic (robust):,3.1984
